In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import os
import pickle as pkl
import glob
import itertools
import xgboost as xgb

#from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, FunctionTransformer
import category_encoders as ce
from sklearn.model_selection import train_test_split

#import da_func
#import data_preprocessing

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
루트경로 = '../../'

데이터경로 = 루트경로 + 'data/'

In [50]:
glob.glob(데이터경로 + 'encoded/*')

['../../data/encoded/binn99_nlohe_nh16_olle_ohle_tmce',
 '../../data/encoded/bin0_nlohe_nh16_olle_ohle_tmle',
 '../../data/encoded/binn99_nlohe_nh16_olle_ohloo_tmle',
 '../../data/encoded/bin0_nlohe_nhloo_olle_ohle_tmce',
 '../../data/encoded/binn99_nlohe_nhloo_olle_ohloo_tmce',
 '../../data/encoded/bin0_nlohe_nhloo_olle_ohloo_tmle',
 '../../data/encoded/binn99_nlohe_nhloo_olle_ohle_tmle',
 '../../data/encoded/bin0_nlohe_nh16_olle_ohloo_tmce',
 '../../data/encoded/binn99_nlloo_nh16_olle_ohloo_tmce',
 '../../data/encoded/bin0_nlloo_nhloo_olle_ohle_tmle',
 '../../data/encoded/bin0_nlloo_nhloo_olle_ohloo_tmle',
 '../../data/encoded/binn99_nlloo_nhloo_olle_ohloo_tmce',
 '../../data/encoded/binn99_nlloo_nh16_olle_ohle_tmce',
 '../../data/encoded/bin0_nlloo_nh16_olle_ohle_tmle',
 '../../data/encoded/binn99_nlloo_nhloo_olle_ohle_tmce',
 '../../data/encoded/bin0_nlloo_nh16_olle_ohloo_tmle',
 '../../data/encoded/bin0_nlohe_nhloo_olle_ohle_tmle',
 '../../data/encoded/binn99_nlohe_nh16_olle_ohle_

In [88]:
# bin0_nlloo_nhloo_olle_ohle_tmle
# bin0_nlohe_nh16_olle_ohle_tmce
# bin0_nlohe_nh16_olle_ohle_tmle
# bin0_nlloo_nh16_olle_ohle_tmce
train_df = pd.read_csv(데이터경로 + 'encoded/bin0_nlloo_nh16_olle_ohle_tmce/' + 'train.csv')
test_df = pd.read_csv(데이터경로 + 'encoded/bin0_nlloo_nh16_olle_ohle_tmce/' + 'test.csv')

In [89]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df.iloc[:, 2:].values, train_df['target'].values,
    test_size=0.2,
    random_state=42
)

In [90]:
y_val

array([0, 0, 0, ..., 0, 1, 0])

In [95]:
xgb_model = xgb.XGBClassifier(
    max_depth=3,
    learning_rate=1e-6,
    n_estimators=500,
    colsample_bytree=0.6,
    reg_alpha=0.5,
    reg_lambda=0.5,
    random_state=42,
    seed=42
)

In [96]:
xgb_model.fit(
    X=X_train,
    y=y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='auc',
    early_stopping_rounds=30,
    #verbose=True,
)

[0]	validation_0-auc:0.950905
Will train until validation_0-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.956964
[2]	validation_0-auc:0.9948
[3]	validation_0-auc:0.992971
[4]	validation_0-auc:0.993774
[5]	validation_0-auc:0.994922
[6]	validation_0-auc:0.994922
[7]	validation_0-auc:0.997243
[8]	validation_0-auc:0.997077
[9]	validation_0-auc:0.999864
[10]	validation_0-auc:0.999663
[11]	validation_0-auc:0.999687
[12]	validation_0-auc:0.999696
[13]	validation_0-auc:0.999765
[14]	validation_0-auc:0.999622
[15]	validation_0-auc:0.999567
[16]	validation_0-auc:0.999813
[17]	validation_0-auc:0.999891
[18]	validation_0-auc:0.999899
[19]	validation_0-auc:0.999897
[20]	validation_0-auc:0.999899
[21]	validation_0-auc:0.999897
[22]	validation_0-auc:0.999893
[23]	validation_0-auc:0.999937
[24]	validation_0-auc:0.999942
[25]	validation_0-auc:0.999938
[26]	validation_0-auc:0.999939
[27]	validation_0-auc:0.999936
[28]	validation_0-auc:0.999938
[29]	validation_0-auc:0.9999
[30]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0,
              learning_rate=1e-06, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0.5, reg_lambda=0.5, scale_pos_weight=1, seed=42,
              silent=None, subsample=1, verbosity=1)

In [97]:
X_test = test_df.iloc[:, 1:].values
#y_test = test_df['targe']
ID_test = test_df[['id']]

In [98]:
xgb_model.predict_proba(X_test)

array([[0.5000178 , 0.49998218],
       [0.5000175 , 0.49998248],
       [0.500017  , 0.499983  ],
       ...,
       [0.50001764, 0.49998236],
       [0.50001746, 0.49998254],
       [0.5000169 , 0.49998307]], dtype=float32)

In [99]:
np.unique(xgb_model.predict(X_test), return_counts=True)

(array([0]), array([400000]))

In [79]:
np.unique(xgb_model.predict(X_val), return_counts=True)

(array([0]), array([120000]))

In [63]:
ID_test['target'] = xgb_model.predict_proba(X_test)[:, 1]

/Users/a60058221/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
ID_test.head()

,id,target
0,600000,0.499961
1,600001,0.499962
2,600002,0.499964
3,600003,0.499962
4,600004,0.499962


In [66]:
ID_test.to_csv(데이터경로 + "org/submission.csv", index=None)